In [2]:
# jupyter notebook 환경에서 spark를 사용하기 위한 모듈
import findspark
findspark.init('C:/spark/spark-3.0.1-bin-hadoop2.7')

In [3]:
# pyspark import
import pyspark
from pyspark import SparkConf,SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql import SparkSession

In [4]:
# pyspark 연결하기
# 꼬이면 kernel restart
conf= pyspark.SparkConf().setAppName('appName').setMaster('local')
sc= pyspark.SparkContext(conf= conf)
spark= SparkSession(sc)

# JSON 데이터 생성하기

In [4]:
stringJSONRDD = sc.parallelize((
"""{"id": "123", "name": "Katie", "age": 19, "eyeColor": "brown"}""",
    """{"id": "234", "name": "Michael", "age": 22, "eyeColor": "green"}""",
    """{"id": "345", "name": "Simone", "age": 23, "eyeColor": "blue"}"""
))

In [6]:
# 데이터프레임 생성하기
swimmersJSON= spark.read.json(stringJSONRDD)

In [8]:
# 임시 테이블 생성하기
swimmersJSON.createOrReplaceTempView("swimmersJSON")

In [10]:
swimmersJSON.show() # dataframe을 콘솔에 출력하는 함수

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



# SQL 쿼리

In [11]:
spark.sql("select * from swimmersJSON").collect()
# collect함수는 모든 행을 리턴하므로 작은 데이터프레임에 대해 사용하는 것이 더 좋다.
# n개의 행을 리턴할 때에는 일반적으로 take(n)이나 show(n)을 사용하는 것이 더 좋다.

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

In [12]:
# 스키마 정의 확인하기
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



In [20]:
# 위 schema에서 명시된 type을 명시하고 싶을 때 사용하는 방법
from pyspark.sql.types import *

stringCSVRDD = sc.parallelize([
    (123, 'Katie', 19, 'brown'),
    (234, 'Michael', 22, 'green'),
    (345, 'Simone', 23, 'blue')
])

# 스키마 명시하기
schema = StructType([
    StructField("id", LongType(), True),
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

In [21]:
# RDD에 스키마를 적용하고 데이터프레임을 생성하기
swimmers = spark.createDataFrame(stringCSVRDD, schema)

# 데이터프레임을 이용해 임시 뷰를 생성
swimmers.createOrReplaceTempView("swimmers")

In [22]:
swimmers.printSchema() # schema에서 명시된 순서와 타입대로 정렬되었음을 알 수 있다.

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



# 데이터프레임 API로 쿼리하기

In [23]:
swimmers.count() # 행 개수 세기

3

In [25]:
# age가 22인 데이터의 id와 age를 출력하기
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [26]:
# 다른 방식
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age == 22).show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [27]:
# eyeColor가 b로 시작하는 데이터의 name, eyeColor 칼럼을 얻는다.
swimmers.select("name", "eyeColor").filter("eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



# SQL 쿼리

In [28]:
spark.sql("select count(1) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



In [29]:
spark.sql("select id, age from swimmers where age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [30]:
spark.sql("select name, eyeColor from swimmers where eyeColor like 'b%'").show() # 눈 색깔이 b로 시작하는 사람 찾기

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



In [5]:
flightPerFilePath = "./dataset/departuredelays.csv"
airportsFilePath = "./dataset/airport-codes-na.txt"

In [6]:
airports = spark.read.csv(airportsFilePath, header = 'true', inferSchema='true', sep='\t')
airports.createOrReplaceTempView("airports")

In [7]:
flightPerf = spark.read.csv(flightPerFilePath, header = 'true')
flightPerf.createOrReplaceTempView("FlightPerformance")

In [8]:
flightPerf.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]